In [2]:
import sklearn
from hmmlearn import hmm
import numpy as np
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from keras.preprocessing.sequence import pad_sequences


C:\Users\212704419\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [350]:
X1 = [[0.5], [1.0], [-1.0], [0.42], [0.24]]
X2 = [[2.4], [4.2], [0.5], [-0.24]]
X3 = np.concatenate([X1, X2])
print(X3)

[[ 0.5 ]
 [ 1.  ]
 [-1.  ]
 [ 0.42]
 [ 0.24]
 [ 2.4 ]
 [ 4.2 ]
 [ 0.5 ]
 [-0.24]]


In [3]:
# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text

In [145]:
# load text
raw_text = load_doc('hs_train.txt')

In [171]:
raw_lines = raw_text.split('\n')
lines = list(map(lambda x: x[x.index('RARITY_END') + len('RARITY_END ')::], raw_lines))

lines = list(filter(lambda x: x != 'NIL', lines))

#lines = raw_lines

lines = list(map(lambda x: x.replace(".", "").replace("<b>", "").replace("</b>", "").replace("</i>", "").replace("<i>", ""), lines))

lines = list(map(lambda x: x.replace("$", "").replace("+", "").replace("(", "").replace(")", "").replace("#", "").replace(":", ""), lines))

#lines = list(map(lambda x: x.replace("nl", ""), lines))

lines = list(map(lambda x: x.lower(), lines))

print(lines)


["battlecry destroy your opponent's weapon", 'restore a minion to full health and give it taunt', 'summon a random beast companion', 'deal 1 damage to all enemy minions', 'draw 2 cards', 'deal 3 damage randomly split among all enemies', 'deal 2 damage', 'destroy an enemy minion', 'deal 2 damage to an undamaged minion', 'give a minion 4/4 4 attack/4 health', 'give a minion 3 attack', 'give your minions 3 attack this turn', 'charge', 'give a friendly minion 2 attack and charge', 'give your hero 2 attack this turn and 2 armor', 'deal 2 damage to two random enemy minions', 'deal 2 damage to all enemies', 'choose an enemy minion at the start of your turn, destroy it', 'spell damage 1', "double a minion's health", 'battlecry summon a 2/1 mechanical dragonling', 'deal 2 damage restore 2 health to your hero', 'battlecry deal 1 damage to all other characters', 'battlecry deal 1 damage', 'draw a card you can only have 10 mana in your tray', 'destroy a damaged enemy minion', 'deal 1 damage to all

In [172]:
#lines = raw_text.split('\n')


maxval = max(list(map(lambda x: len(x), lines)))

minval = min(list(map(lambda x: len(x), lines)))

padded_lines = list(map(lambda x: (x + ('*' * (maxval - len(x)))) if (len(x) < maxval) else x, lines))

chopped_lines = list(map(lambda x: x[: (minval)] if (len(x) > minval) else x, lines))


flat_list = [item for sublist in padded_lines for item in sublist]

chars = sorted(list(set(flat_list)))

mapping = dict((c, i) for i, c in enumerate(chars))

print(mapping)


words = list(map(lambda x: x.split(" "), lines))

#words = list(filter(lambda x: len(x) > 6 and len(x) < 12, words))

maxvalwords = max(list(map(lambda x: len(x), words)))
minvalwords = min(list(map(lambda x: len(x), words)))


padded_words = list(map(lambda x: (x + (['blank'] * (maxvalwords - len(x)))) if (len(x) < maxvalwords) else x, words))

chopped_words = chopped_lines = list(map(lambda x: x[: (minvalwords)] if (len(x) > minvalwords) else x, words))

print(np.mean(np.array(list(map(lambda x: len(x), words)))))

flat_list_words = [item for sublist in words for item in sublist]

all_words = sorted(list(set(flat_list_words)))
word_map = dict((c, i) for i, c in enumerate(all_words))


{' ': 0, '!': 1, '"': 2, '%': 3, "'": 4, '*': 5, ',': 6, '-': 7, '/': 8, '0': 9, '1': 10, '2': 11, '3': 12, '4': 13, '5': 14, '6': 15, '7': 16, '8': 17, ';': 18, 'a': 19, 'b': 20, 'c': 21, 'd': 22, 'e': 23, 'f': 24, 'g': 25, 'h': 26, 'i': 27, 'j': 28, 'k': 29, 'l': 30, 'm': 31, 'n': 32, 'o': 33, 'p': 34, 'q': 35, 'r': 36, 's': 37, 't': 38, 'u': 39, 'v': 40, 'w': 41, 'x': 42, 'y': 43, 'z': 44, 'â': 45, '€': 46, '™': 47}
8.227016885553471


In [173]:
sequences = list()
for line in padded_lines:
    # integer encode line
    encoded_seq = [mapping[char] for char in line]
    # store
    sequences.append((np.array(encoded_seq)))

# vocabulary size
vocab_size = len(mapping)
print('Vocabulary Size: %d' % vocab_size)

Vocabulary Size: 48


In [174]:
temp = []
lengths_arr = []
for line in words:
    # integer encode line
    encoded_seq = ([np.array(word_map[word]) for word in line])
    lengths_arr.append(len(encoded_seq))
    # store
    temp += encoded_seq

# vocabulary size
vocab_size = len(mapping)
print(vocab_size)

48


In [175]:
sequences = np.array(temp)

print(sequences)

[ 79 147 422 ... 149 380 393]


In [176]:
print(sequences.shape)

(4385,)


In [177]:
sequences = (np.array(sequences.reshape(-1,1)))
hmm_model = hmm.MultinomialHMM(n_components=6)
hmm_model.fit(sequences, lengths_arr)

MultinomialHMM(algorithm='viterbi', init_params='ste', n_components=6,
        n_iter=10, params='ste',
        random_state=<mtrand.RandomState object at 0x00000234380D9CF0>,
        startprob_prior=1.0, tol=0.01, transmat_prior=1.0, verbose=False)

In [178]:
predictions = hmm_model.sample(20)

inv_map = {v: k for k, v in word_map.items()}

output = ""

for char_arr in predictions[0]:
    
    output += inv_map[char_arr[0]] + " "

print(output)
#print(output.split("."))

nl damage 5/2 replace hero totems hero 2 to to remove if 4 characters gain minions at 1 a whenever 


In [179]:
joined_lines  = " ".join(lines)

length = 12
sequences = list()
for line in lines:
    for i in range(length, len(line)):
        # select sequence of tokens
        seq = line[i-length:i+1]
        # store
        sequences.append(seq)
print('Total Sequences: %d' % len(sequences))

new_input = sequences

Total Sequences: 17061


In [180]:
print(new_input[:20])

['battlecry des', 'attlecry dest', 'ttlecry destr', 'tlecry destro', 'lecry destroy', 'ecry destroy ', 'cry destroy y', 'ry destroy yo', 'y destroy you', ' destroy your', 'destroy your ', 'estroy your o', 'stroy your op', 'troy your opp', 'roy your oppo', 'oy your oppon', 'y your oppone', ' your opponen', 'your opponent', "our opponent'"]


In [181]:
chars = sorted(list(set(joined_lines)))
mapping = dict((c, i) for i, c in enumerate(chars))

vocab_size = len(mapping)

sequences = list()
for line in new_input:
	# integer encode line
	encoded_seq = [mapping[char] for char in line]
	# store
	sequences.append(np.array(encoded_seq))

In [182]:
sequences = np.array(sequences)

print(sequences.shape)
np.random.shuffle(sequences)
X, y = sequences[:,:-1], sequences[:,-1]

(17061, 13)


In [183]:
sequences = [to_categorical(x, num_classes=vocab_size) for x in X]


X = np.array(sequences)
y = to_categorical(y, num_classes=vocab_size)

In [184]:
print(y)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [185]:
# define model
n_batch = len(X)
model = Sequential()
model.add(LSTM(128, input_shape=( X.shape[1], X.shape[2]),return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(128))
model.add(Dropout(0.2))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_21 (LSTM)               (None, 12, 128)           90112     
_________________________________________________________________
dropout_17 (Dropout)         (None, 12, 128)           0         
_________________________________________________________________
lstm_22 (LSTM)               (None, 128)               131584    
_________________________________________________________________
dropout_18 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 47)                6063      
Total params: 227,759
Trainable params: 227,759
Non-trainable params: 0
_________________________________________________________________
None


In [186]:
# compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit model
model.fit(X, y, epochs=100, verbose=2)


Epoch 1/100
 - 20s - loss: 2.8835 - acc: 0.2053
Epoch 2/100
 - 17s - loss: 2.1121 - acc: 0.4099
Epoch 3/100
 - 18s - loss: 1.6128 - acc: 0.5531
Epoch 4/100
 - 17s - loss: 1.3386 - acc: 0.6266
Epoch 5/100
 - 18s - loss: 1.1679 - acc: 0.6756
Epoch 6/100
 - 17s - loss: 1.0454 - acc: 0.7046
Epoch 7/100
 - 17s - loss: 0.9381 - acc: 0.7310
Epoch 8/100
 - 18s - loss: 0.8671 - acc: 0.7522
Epoch 9/100
 - 17s - loss: 0.7981 - acc: 0.7697
Epoch 10/100
 - 18s - loss: 0.7470 - acc: 0.7815
Epoch 11/100
 - 18s - loss: 0.6933 - acc: 0.7989
Epoch 12/100
 - 17s - loss: 0.6585 - acc: 0.8043
Epoch 13/100
 - 18s - loss: 0.6186 - acc: 0.8157
Epoch 14/100
 - 18s - loss: 0.5840 - acc: 0.8245
Epoch 15/100
 - 18s - loss: 0.5564 - acc: 0.8313
Epoch 16/100
 - 17s - loss: 0.5243 - acc: 0.8389
Epoch 17/100
 - 18s - loss: 0.4973 - acc: 0.8496
Epoch 18/100
 - 17s - loss: 0.4770 - acc: 0.8542
Epoch 19/100
 - 18s - loss: 0.4602 - acc: 0.8592
Epoch 20/100
 - 17s - loss: 0.4358 - acc: 0.8659
Epoch 21/100
 - 18s - loss: 0

In [36]:
n_epoch = 1000
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit network
for i in range(n_epoch):
	model.fit(X, y, epochs=1, batch_size=n_batch, verbose=1, shuffle=False)
	model.reset_states()

Epoch 1/1
14039/14039 [==============================] - 5s 374us/step - loss: 0.1441 - acc: 0.9509
Epoch 1/1
14039/14039 [==============================] - 4s 294us/step - loss: 0.1321 - acc: 0.9556
Epoch 1/1
14039/14039 [==============================] - 4s 290us/step - loss: 0.1278 - acc: 0.9561
Epoch 1/1
14039/14039 [==============================] - 4s 285us/step - loss: 0.1259 - acc: 0.9575
Epoch 1/1
14039/14039 [==============================] - 4s 287us/step - loss: 0.1264 - acc: 0.9575
Epoch 1/1
14039/14039 [==============================] - 4s 311us/step - loss: 0.1198 - acc: 0.9583
Epoch 1/1
14039/14039 [==============================] - 5s 323us/step - loss: 0.1156 - acc: 0.9596
Epoch 1/1
14039/14039 [==============================] - 4s 309us/step - loss: 0.1199 - acc: 0.9582
Epoch 1/1
14039/14039 [==============================] - 4s 320us/step - loss: 0.1170 - acc: 0.9577
Epoch 1/1
14039/14039 [==============================] - 4s 311us/step - loss: 0.1161 - acc: 0.9580


KeyboardInterrupt: 

In [226]:
# generate a sequence of characters with a language model
def generate_seq(model, mapping, seq_length, seed_text, n_chars):
    in_text = seed_text
    # generate a fixed number of characters
    for _ in range(n_chars):
        # encode the characters as integers
        encoded = [mapping[char] for char in in_text]
        # truncate sequences to a fixed length
        encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
        # one hot encode
        
        encoded = to_categorical(encoded, num_classes=len(mapping))
    
        encoded = encoded.reshape(1, encoded.shape[1], encoded.shape[2])
        # predict character
        yhat = model.predict_classes(encoded, verbose=0)
        # reverse map integer to character
        out_char = ''
        for char, index in mapping.items():
            if index == yhat:
                out_char = char
                break
        # append to input
        in_text += char
    return in_text

# test start of rhyme
#print(generate_seq(model, mapping, 10, 'battlecry:', 20))
# test mid-linery
text = "choose one -"
print(generate_seq(model, mapping, 12, text, 400))
# test not in original
#print(generate_seq(model, mapping, 10, 'hello worl', 20))

choose one - give your other minions have 1 attack and health of all minions to their owner's hand and it costs 2 less benacen and freeze them attack of your weapon has 2 attack while you have a mech, gain divine shield gain 3/3 for each shield lost dis attack damage to all enemies restore 2 health to your hero and gain 2 attack this turn and 4 armor nl draw a card you can only have 15 s conts to your hand it


In [ ]:
battlecry destroy a murloc and gain 2/2 for each card in your hand

give a minion 2 attack this turn and 4 armor

divine a random 4-cost minion for your opponent casts a spell

costs 1 less for each minion that died this game, summon thaddius

deal 2 damage to all enemies restore 2 health to your hero and gain 2 attack this turn

choose one - give your other minions have 1 attack and health